In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data1 = pd.read_csv("data/LoanStats_securev1_2018Q4.csv",skiprows=1)

C:\Users\zachv\.conda\envs\py38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data1.shape

(128414, 150)

In [4]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.float_format', "{:,.2f}".format)
data1.dtypes.sort_values()

total_rev_hi_lim                              float64
num_op_rev_tl                                 float64
num_il_tl                                     float64
num_bc_tl                                     float64
num_bc_sats                                   float64
num_actv_rev_tl                               float64
num_actv_bc_tl                                float64
num_accts_ever_120_pd                         float64
mths_since_recent_revol_delinq                float64
mths_since_recent_inq                         float64
mths_since_recent_bc_dlq                      float64
mths_since_recent_bc                          float64
mort_acc                                      float64
mo_sin_rcnt_tl                                float64
mo_sin_rcnt_rev_tl_op                         float64
num_rev_accts                                 float64
mo_sin_old_rev_tl_op                          float64
delinq_amnt                                   float64
chargeoff_within_12_mths    

In [5]:
#Looking to cut by grade:
data1.grade.value_counts(dropna=False)

A      38011
B      35861
C      31018
D      17072
E       5962
F        387
G        101
NaN        2
Name: grade, dtype: int64

In [6]:
# we have 2 NA grades:
data1[pd.isna(data1.grade)].T

,128412,128413
id,Total amount funded in policy code 1: 2050909275,Total amount funded in policy code 2: 820109297
member_id,NaN,NaN
loan_amnt,NaN,NaN
funded_amnt,NaN,NaN
funded_amnt_inv,NaN,NaN
term,NaN,NaN
int_rate,NaN,NaN
installment,NaN,NaN
grade,NaN,NaN
sub_grade,NaN,NaN


In [7]:
# They're just footnotes with no values for all other columns
data2 = data1[~pd.isna(data1.grade)].copy()

In [8]:
#we want to put F and G together
data2["grade2"] = data2["grade"]
data2.loc[data1["grade"].isin(["F","G"]), "grade2"] = "FG"
data2.grade2.value_counts(dropna=False)

A     38011
B     35861
C     31018
D     17072
E      5962
FG      488
Name: grade2, dtype: int64

In [9]:
final_table = pd.DataFrame(index=data2.grade2.unique())
final_table = final_table.sort_index()
final_table

""
A
B
C
D
E
FG


In [10]:
data2.groupby("grade2").loan_amnt.sum()

grade2
A    604,816,200.00
B    591,596,125.00
C    495,916,925.00
D    267,034,700.00
E     83,832,000.00
FG     7,713,325.00
Name: loan_amnt, dtype: float64

In [11]:
final_table["Total Issued"] = data2.groupby("grade2").loan_amnt.sum()
final_table

,Total Issued
A,"604,816,200.00"
B,"591,596,125.00"
C,"495,916,925.00"
D,"267,034,700.00"
E,"83,832,000.00"
FG,"7,713,325.00"


In [12]:
data2.loan_status.value_counts()

Current               94805
Fully Paid            23573
Charged Off            6071
Late (31-120 days)     2582
In Grace Period         881
Late (16-30 days)       496
Default                   4
Name: loan_status, dtype: int64

In [13]:
data2[data2.loan_status == "Fully Paid"].groupby("grade2").loan_amnt.sum()

grade2
A    100,618,700.00
B     93,436,350.00
C     80,649,100.00
D     44,926,850.00
E     16,174,675.00
FG     1,668,325.00
Name: loan_amnt, dtype: float64

In [14]:
final_table["Fully Paid"] = data2[data2.loan_status == "Fully Paid"].groupby("grade2").loan_amnt.sum()
final_table

,Total Issued,Fully Paid
A,"604,816,200.00","100,618,700.00"
B,"591,596,125.00","93,436,350.00"
C,"495,916,925.00","80,649,100.00"
D,"267,034,700.00","44,926,850.00"
E,"83,832,000.00","16,174,675.00"
FG,"7,713,325.00","1,668,325.00"


In [15]:
data2[data2.loan_status == "Current"].groupby("grade2").loan_amnt.sum()

grade2
A    485,245,175.00
B    459,702,350.00
C    362,191,200.00
D    180,795,200.00
E     49,523,000.00
FG     3,906,550.00
Name: loan_amnt, dtype: float64

In [16]:
#We likely need the current amount
data2[data2.loan_status == "Current"].groupby("grade2").out_prncp.sum()

grade2
A    319,179,923.70
B    333,238,892.66
C    270,755,833.64
D    139,940,139.46
E     38,770,773.58
FG     3,272,217.70
Name: out_prncp, dtype: float64

In [17]:
#A little short, maybe using grace period loans as well?
data2[data2.loan_status.isin(["Current","In Grace Period"])].groupby("grade2").out_prncp.sum()

grade2
A    320,372,435.11
B    336,035,184.56
C    274,760,919.88
D    142,663,648.82
E     39,733,448.06
FG     3,337,915.23
Name: out_prncp, dtype: float64

In [18]:
#off by a few dollars, check a few columns to see if they could be the difference
data2[data2.loan_status.isin(["Current","In Grace Period"])].groupby("grade2").delinq_amnt.sum()

grade2
A    13,854.00
B     4,649.00
C    33,957.00
D    12,589.00
E     5,739.00
FG        0.00
Name: delinq_amnt, dtype: float64

In [19]:
data2[data2.loan_status.isin(["Current","In Grace Period"])].groupby("grade2").hardship_amount.sum()

grade2
A      985.45
B    3,472.42
C    6,456.38
D    7,066.51
E    2,668.15
FG       0.00
Name: hardship_amount, dtype: float64

In [20]:
#likely just a rounding error
final_table["Current"] = data2[data2.loan_status.isin(["Current","In Grace Period"])].groupby("grade2").out_prncp.sum()
final_table

,Total Issued,Fully Paid,Current
A,"604,816,200.00","100,618,700.00","320,372,435.11"
B,"591,596,125.00","93,436,350.00","336,035,184.56"
C,"495,916,925.00","80,649,100.00","274,760,919.88"
D,"267,034,700.00","44,926,850.00","142,663,648.82"
E,"83,832,000.00","16,174,675.00","39,733,448.06"
FG,"7,713,325.00","1,668,325.00","3,337,915.23"


In [21]:
data2[data2.loan_status.str.contains("Late")].groupby("grade2").out_prncp.sum()

grade2
A     5,134,582.88
B     9,173,916.60
C    13,022,871.87
D     9,785,213.67
E     4,326,428.61
FG      376,399.95
Name: out_prncp, dtype: float64

In [22]:
# lower rated loans a bit worse, see if late includes defaulted as well
data2[data2.loan_status.str.contains("Late") | (data2.loan_status == "Default")].groupby("grade2").out_prncp.sum()

grade2
A     5,134,582.88
B     9,179,111.00
C    13,036,225.30
D     9,795,590.90
E     4,326,428.61
FG      378,755.61
Name: out_prncp, dtype: float64

In [23]:
final_table["Late"] = data2[data2.loan_status.str.contains("Late") | (data2.loan_status == "Default")].groupby("grade2").out_prncp.sum()
final_table

,Total Issued,Fully Paid,Current,Late
A,"604,816,200.00","100,618,700.00","320,372,435.11","5,134,582.88"
B,"591,596,125.00","93,436,350.00","336,035,184.56","9,179,111.00"
C,"495,916,925.00","80,649,100.00","274,760,919.88","13,036,225.30"
D,"267,034,700.00","44,926,850.00","142,663,648.82","9,795,590.90"
E,"83,832,000.00","16,174,675.00","39,733,448.06","4,326,428.61"
FG,"7,713,325.00","1,668,325.00","3,337,915.23","378,755.61"


In [24]:
data2[data2.loan_status == "Charged Off"].groupby("grade2").loan_amnt.sum()

grade2
A    10,083,075.00
B    22,570,550.00
C    31,491,025.00
D    25,847,775.00
E    11,813,950.00
FG    1,611,225.00
Name: loan_amnt, dtype: float64

In [25]:
data2[data2.loan_status == "Charged Off"].groupby("grade2").loan_amnt.sum() \
- data2[data2.loan_status == "Charged Off"].groupby("grade2").total_pymnt.sum() \
+ data2[data2.loan_status == "Charged Off"].groupby("grade2").total_rec_int.sum() \
+ data2[data2.loan_status == "Charged Off"].groupby("grade2").total_rec_late_fee.sum()

grade2
A     8,463,972.13
B    19,315,342.01
C    26,937,908.13
D    22,203,105.49
E    10,433,630.56
FG    1,418,711.82
dtype: float64

In [26]:
#Grades A, D, and E are matching, the rest are under expected by ~1-2k
data2[data2.loan_status == "Charged Off"].groupby("grade2").collection_recovery_fee.sum()

grade2
A     29,567.78
B    113,764.24
C    247,647.34
D    260,008.48
E     98,248.46
FG    17,488.67
Name: collection_recovery_fee, dtype: float64

In [27]:
# are there any numeric fields where A, D, and E are very low, while B, C, and FG are in the expected range?
data2[data2.loan_status == "Charged Off"].groupby("grade2").sum().T.sort_values("A")

grade2,A,B,C,D,E,FG
member_id,0.00,0.00,0.00,0.00,0.00,0.00
acc_now_delinq,0.00,0.00,0.00,0.00,0.00,0.00
delinq_amnt,0.00,0.00,0.00,"31,593.00",0.00,0.00
num_tl_120dpd_2m,0.00,0.00,0.00,0.00,0.00,0.00
num_tl_30dpd,0.00,0.00,0.00,0.00,0.00,0.00
out_prncp_inv,0.00,0.00,0.00,0.00,0.00,0.00
out_prncp,0.00,0.00,0.00,0.00,0.00,0.00
sec_app_chargeoff_within_12_mths,0.00,5.00,11.00,11.00,8.00,2.00
tax_liens,0.00,0.00,0.00,0.00,0.00,0.00
desc,0.00,0.00,0.00,0.00,0.00,0.00


In [28]:
# Looked back at non-numeric fields, potential factors could be hardship or payment plans
data2[data2.loan_status == "Charged Off"].groupby("grade2").settlement_status.value_counts()

grade2  settlement_status
A       ACTIVE                74
        COMPLETE               7
        BROKEN                 2
B       ACTIVE               126
        COMPLETE              11
        BROKEN                 9
C       ACTIVE               113
        COMPLETE               9
        BROKEN                 3
D       ACTIVE                65
        BROKEN                10
        COMPLETE               8
E       ACTIVE                53
        BROKEN                 7
        COMPLETE               5
FG      ACTIVE                 2
Name: settlement_status, dtype: int64

In [29]:
data2[data2.loan_status == "Charged Off"].groupby("grade2").hardship_status.value_counts()

grade2  hardship_status
A       BROKEN             4
B       BROKEN             6
C       BROKEN             5
D       BROKEN             6
E       BROKEN             1
Name: hardship_status, dtype: int64

In [30]:
# maybe one of the fields in the calculation are missing for some and there's an imputation I'm not aware of
pd.isna(data2[data2.loan_status == "Charged Off"][["loan_amnt","total_pymnt","total_rec_int","total_rec_late_fee"]]).sum()

loan_amnt             0
total_pymnt           0
total_rec_int         0
total_rec_late_fee    0
dtype: int64

In [31]:
# ok we'll settle for this calculation and can follow up on what's 
# included in net charge off to see if there's something we're missing
final_table["Charged Off (Net)"] = \
data2[data2.loan_status == "Charged Off"].groupby("grade2").loan_amnt.sum() \
- data2[data2.loan_status == "Charged Off"].groupby("grade2").total_pymnt.sum() \
+ data2[data2.loan_status == "Charged Off"].groupby("grade2").total_rec_int.sum() \
+ data2[data2.loan_status == "Charged Off"].groupby("grade2").total_rec_late_fee.sum()
final_table

,Total Issued,Fully Paid,Current,Late,Charged Off (Net)
A,"604,816,200.00","100,618,700.00","320,372,435.11","5,134,582.88","8,463,972.13"
B,"591,596,125.00","93,436,350.00","336,035,184.56","9,179,111.00","19,315,342.01"
C,"495,916,925.00","80,649,100.00","274,760,919.88","13,036,225.30","26,937,908.13"
D,"267,034,700.00","44,926,850.00","142,663,648.82","9,795,590.90","22,203,105.49"
E,"83,832,000.00","16,174,675.00","39,733,448.06","4,326,428.61","10,433,630.56"
FG,"7,713,325.00","1,668,325.00","3,337,915.23","378,755.61","1,418,711.82"


In [32]:
data2.groupby("grade2").total_rec_prncp.sum().round()

grade2
A    270,680,944.00
B    226,434,464.00
C    179,806,053.00
D     90,927,863.00
E     28,792,668.00
FG     2,480,783.00
Name: total_rec_prncp, dtype: float64

In [33]:
# Last calculation likely missing edge cases, off by a lot. This calculation is correct based on final expected table
final_table["Principal Payments Received"] = \
final_table["Total Issued"] - \
final_table["Current"] - \
final_table["Late"] - \
final_table["Charged Off (Net)"]
final_table

,Total Issued,Fully Paid,Current,Late,Charged Off (Net),Principal Payments Received
A,"604,816,200.00","100,618,700.00","320,372,435.11","5,134,582.88","8,463,972.13","270,845,209.88"
B,"591,596,125.00","93,436,350.00","336,035,184.56","9,179,111.00","19,315,342.01","227,066,487.43"
C,"495,916,925.00","80,649,100.00","274,760,919.88","13,036,225.30","26,937,908.13","181,181,871.69"
D,"267,034,700.00","44,926,850.00","142,663,648.82","9,795,590.90","22,203,105.49","92,372,354.79"
E,"83,832,000.00","16,174,675.00","39,733,448.06","4,326,428.61","10,433,630.56","29,338,492.77"
FG,"7,713,325.00","1,668,325.00","3,337,915.23","378,755.61","1,418,711.82","2,577,942.34"


In [34]:
data2.groupby("grade2").total_rec_int.sum().round()

grade2
A    40,954,995.00
B    62,120,597.00
C    68,739,559.00
D    47,314,655.00
E    18,142,306.00
FG    1,947,021.00
Name: total_rec_int, dtype: float64

In [35]:
# Same grades match as with charge offs, still likely an edge case causing small differences
final_table["Interest Payments Received"] = data2.groupby("grade2").total_rec_int.sum()
final_table

,Total Issued,Fully Paid,Current,Late,Charged Off (Net),Principal Payments Received,Interest Payments Received
A,"604,816,200.00","100,618,700.00","320,372,435.11","5,134,582.88","8,463,972.13","270,845,209.88","40,954,995.43"
B,"591,596,125.00","93,436,350.00","336,035,184.56","9,179,111.00","19,315,342.01","227,066,487.43","62,120,597.03"
C,"495,916,925.00","80,649,100.00","274,760,919.88","13,036,225.30","26,937,908.13","181,181,871.69","68,739,558.84"
D,"267,034,700.00","44,926,850.00","142,663,648.82","9,795,590.90","22,203,105.49","92,372,354.79","47,314,654.95"
E,"83,832,000.00","16,174,675.00","39,733,448.06","4,326,428.61","10,433,630.56","29,338,492.77","18,142,305.80"
FG,"7,713,325.00","1,668,325.00","3,337,915.23","378,755.61","1,418,711.82","2,577,942.34","1,947,021.11"


In [36]:
data2.int_rate

0          14.47%
1          22.35%
2          23.40%
3          18.94%
4          10.72%
           ...   
128407     15.02%
128408     15.02%
128409     13.56%
128410     11.06%
128411     16.91%
Name: int_rate, Length: 128412, dtype: object

In [37]:
# need to clean to floats
data2["int_rate_float"] = data2.int_rate.str.strip("%").astype(float) / 100
print(data2.int_rate_float.min())
print(data2.int_rate_float.max())

0.06
0.3099


In [38]:
# assuming average interest rate would be balance-weighted
data2["avg_int_numerator"] = data2["int_rate_float"] * data2["loan_amnt"]
data2.groupby("grade2").avg_int_numerator.sum() / data2.groupby("grade2").loan_amnt.sum()

grade2
A    0.07
B    0.11
C    0.15
D    0.20
E    0.25
FG   0.30
dtype: float64

In [39]:
final_table["Avg. Interest Rate"] = data2.groupby("grade2").avg_int_numerator.sum() / data2.groupby("grade2").loan_amnt.sum()
final_table

,Total Issued,Fully Paid,Current,Late,Charged Off (Net),Principal Payments Received,Interest Payments Received,Avg. Interest Rate
A,"604,816,200.00","100,618,700.00","320,372,435.11","5,134,582.88","8,463,972.13","270,845,209.88","40,954,995.43",0.07
B,"591,596,125.00","93,436,350.00","336,035,184.56","9,179,111.00","19,315,342.01","227,066,487.43","62,120,597.03",0.11
C,"495,916,925.00","80,649,100.00","274,760,919.88","13,036,225.30","26,937,908.13","181,181,871.69","68,739,558.84",0.15
D,"267,034,700.00","44,926,850.00","142,663,648.82","9,795,590.90","22,203,105.49","92,372,354.79","47,314,654.95",0.20
E,"83,832,000.00","16,174,675.00","39,733,448.06","4,326,428.61","10,433,630.56","29,338,492.77","18,142,305.80",0.25
FG,"7,713,325.00","1,668,325.00","3,337,915.23","378,755.61","1,418,711.82","2,577,942.34","1,947,021.11",0.30
